In [9]:
import numpy as np
import tensorflow as tf
import cv2
import keras
from keras import layers
from keras import optimizers
from keras.layers import Input, Lambda, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, AlphaDropout
from keras.layers import Conv2D, SpatialDropout2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D, GlobalMaxPooling2D, GlobalMaxPooling2D, Dropout
from keras.models import Model, load_model
from keras.callbacks import ModelCheckpoint, LearningRateScheduler, ReduceLROnPlateau
from keras.utils import plot_model
import keras.backend as K
from keras.applications import resnet, resnet_v2, vgg16, vgg19
from keras.datasets import mnist
from keras.utils import to_categorical
import pandas as pd

In [4]:
(x_tr, y_tr), (x_test_norm, y_test_norm) = mnist.load_data()
x_tr = np.asarray([cv2.resize(a, (56, 56), interpolation = cv2.INTER_LINEAR) for a in x_tr])
x_test_norm = np.asarray([cv2.resize(a, (56, 56), interpolation = cv2.INTER_LINEAR) for a in x_test_norm])

In [5]:
x_tr = x_tr.reshape(60000, 56, 56, 1)
x_test_norm = x_test_norm.reshape(10000, 56, 56, 1)
y_tr = to_categorical(y_tr)
y_test_norm = to_categorical(y_test_norm)

In [6]:
base_model = vgg19.VGG19(include_top = False, input_shape = (56, 56, 1), weights = None, pooling='none')

last = base_model.layers[-1]

x = Flatten()(last.output)
x = Dense(4096, activation = 'relu')(x)
x = Dropout(0.5)(x)
x = Dense(4096, activation = 'relu')(x)
x = Dropout(0.5)(x)

x = Dense(10, activation = 'softmax')(x)

model = Model(inputs=base_model.input, outputs=x)

In [29]:
model.summary()

Model: "model_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_6 (InputLayer)         (None, 56, 56, 1)         0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 56, 56, 64)        640       
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 56, 56, 64)        36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 28, 28, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 28, 28, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 28, 28, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 14, 14, 128)       0   

In [32]:
optimizer = keras.optimizers.SGD(lr=0.001, momentum = 0.9, nesterov = True)
model.compile(optimizer, loss=keras.losses.categorical_crossentropy, metrics=['accuracy'])

In [ ]:
model.fit(x_tr, y_tr, validation_data = (x_test_norm, y_test_norm), batch_size=32, epochs=2)

In [23]:
attacktest = pd.read_csv('whitetoblack.csv', header=None)
attacktest = attacktest.to_numpy()
attacktest = attacktest.reshape(10000, 56, 56, 1)
model.evaluate(attacktest, y_test_norm)

In [20]:
model.evaluate(x_test_norm, y_test_norm)

10000/10000 [==============================] - 7s 729us/step


[0.02307671256269241, 0.993399977684021]

In [1]:
# model.save('models/VGG19_9934.h5')

NameError: name 'model' is not defined

In [7]:
model = load_model('models/VGG19_9934.h5')